In [42]:
import pandas as pd
import os
import numpy as np


In [39]:
with open("search_terms.txt","r") as f:
    file=[line.replace("\n","").strip() for line in f.readlines()]

file


['Voice over english uk',
 'Voice over eng usa',
 'Male voice over',
 'Female',
 'Young adult voice over',
 'French blog post',
 'Morgan Freeman',
 'Child voice for videogame character',
 'Elegant man voice for advertising in english',
 'Female commercial voice in Spanish',
 'Phone system female voice actor',
 'Animations 2D characters',
 'Icons for Facebook',
 'Blog for government',
 'Social media post',
 'Geology',
 'Funny blogpost',
 'Ads video',
 'Piano music',
 'Voice for radio advertisement']

In [21]:
import nltk
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jhonp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jhonp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [246]:
import spacy
nlp = spacy.load('en_core_web_sm')

stopwords = nlp.Defaults.stop_words

In [256]:
import re 
finder=re.compile(pattern='[^A-Za-z0-9 ]+')
lemmatizer = WordNetLemmatizer()


def remove_special_characters(sentence: str,finder=finder,replacement=""):
    return re.sub(finder,replacement,sentence.lower())


def clean_sentence(sentence: str,finder=finder,lemmatizer=lemmatizer):
    """
    lemmatize, remove special characters, accents and standardize a sentence
    """

    sentence=re.sub("([A-Z])", r' \1', sentence) # separate words by uppercase letters eg. telephoneNumber -> telephone Number
    sentence=remove_special_characters(sentence,finder=finder)
    sentence=[lemmatizer.lemmatize(word) for word in sentence.split(" ")]  #lemmatize by noun
    sentence=[word for word in sentence if word not in stopwords or word=="not"] # remove stopword, however negation matters!
    return " ".join(sentence).strip()

clean_sentence("i am in a catedral using not have house")

'catedral not house'

In [49]:
def augment_sample(id,category):
    pass

audio      602586
article     22968
video        3692
image        2838
Name: category, dtype: int64

In [302]:
pro_stats=pd.read_csv("result/pro_stats.csv")
sample_attributes=pd.read_csv("result/sample_attributes.csv")
sample_tags=pd.read_csv("result/sample_tags.csv")
print(pro_stats.shape)
pro_stats.head()

(108140, 10)


,pro_id,bookings,expired_samples,samples_rejected_internally,speed_to_book,average_review,num_favorites,category,successful_bookings,successful_projects
0,A,0,0,0,NaN,NaN,0,audio,0,0
1,E,0,0,0,NaN,NaN,0,audio,0,0
2,13,0,0,0,NaN,NaN,0,audio,0,0
3,18,0,0,0,NaN,NaN,0,audio,0,0
4,1637D,0,0,0,NaN,NaN,0,audio,0,0


In [305]:
sample_attributes.query("sample_id==524487")

,sample_id,category,pro_id,booking_score,attribute_name,attribute_value
631188,524487,article,26F1C9,0.0,voice,first-person
631189,524487,article,26F1C9,0.0,tone_serious_humorous,0.2
631190,524487,article,26F1C9,0.0,tone_concise_wordy,0
631191,524487,article,26F1C9,0.0,tone_objective_opinionated,0.2
631192,524487,article,26F1C9,0.0,tone_sincere_sarcastic,0
631193,524487,article,26F1C9,0.0,tone_journalistic_creative,0.2
631194,524487,article,26F1C9,0.0,article_language_id,eng-us
631195,524487,article,26F1C9,0.0,keywords,"writing,motivational,self-help"
631196,524487,article,26F1C9,0.0,article_structure,any


In [ ]:
##make descriptive plot to show how every feature per category was selected
sample_attributes.query('category=="article"').attribute_name.value_counts()

In [193]:
def to_float(text):
    try:
        value=float(text)
        return value
    except:
        return text

In [293]:
sample_attributes.fillna("",inplace=True)
sample_attributes.loc[:,'if_article_specs']=sample_attributes.apply(lambda k: k['attribute_name'].split("_")[-1] if k['attribute_name'] in ['tone_serious_humorous','tone_sincere_sarcastic','tone_journalistic_creative'] and to_float(k['attribute_value'])>0 else "",axis=1)
sample_attributes.head()

,sample_id,category,pro_id,booking_score,attribute_name,attribute_value,if_article_specs
0,7,audio,D6,0.0,purpose,phoneSystems,
1,7,audio,D6,0.0,language_id,eng-us,
2,7,audio,D6,0.0,gender_and_age_id,youngAdultFemale,
3,21,audio,2362,0.0,purpose,phoneSystems,
4,21,audio,2362,0.0,language_id,eng-us,


In [125]:
sample_attributes.category.value_counts()

audio      602586
article     22968
video        3692
image        2838
Name: category, dtype: int64

In [294]:
attributes_by_category={
    "audio":['purpose','language_id','gender_and_age_id'],
    "video": ['sample_language','purpose','keywords'],
    "image": ['sample_language',"sample_purpose","sample_service"],
    "article": ['article_language_id','keywords','if_article_specs']
    }

In [295]:
rows_to_keep=sample_attributes.apply(lambda k: True if k['attribute_name'] in attributes_by_category[k['category']] else False, axis=1)

In [296]:
## new sample_attributes file
sample_attributes=sample_attributes.loc[rows_to_keep,:]

In [297]:
##some hard-coded examples for this test

REPLACEMENT_LANGUAGES={

    'eng':'english',
    'spa': 'spanish',
    'fra': 'french',
    'por': 'portuguese',
    'deu': 'german',
    'jpn': 'japanese',
    'kor':'korean',
    'tur':'turkish',
    'dan':'danish',
    'latam':'latino',
    'uk': 'british',
    'fr': 'france',
    'us':'american'}


def decode_language_code(lang_code:str,decoder_dict=REPLACEMENT_LANGUAGES):
    lang_code=lang_code.lower().split("-")
    lang_code=[word if word not in decoder_dict.keys() else decoder_dict[word] for word in lang_code]
    return " ".join(lang_code)

decode_language_code("eng-us")

'english american'

In [298]:
#modify attributes value to be more specific
sample_attributes.loc[:,'attribute_value']=sample_attributes.apply(lambda d: clean_sentence(str(d['attribute_value'])) if "language" not in d['attribute_name'] else decode_language_code(str(d['attribute_value'])),axis=1)

In [299]:
#concatenate attribute_value in one single column
sample_attributes.loc[:,'attribute_value']=sample_attributes.attribute_value+""+sample_attributes.if_article_specs

In [301]:
sample_attributes.groupby(['sample_id','category'])['attribute_value'].agg(lambda d: " ".join(list(set(d)))).reset_index().query("category=='article'")

,sample_id,category,attribute_value
78048,227439,article,onetwothree english american
78123,227621,article,social medium blog traffic googleranking engli...
78236,227899,article,english american
78738,229077,article,english american
78946,229567,article,english american
...,...,...,...
205013,524487,article,writingmotivationalselfhelp english american
205131,524762,article,mental healthcovid19frontline worker english a...
205132,524764,article,english american
205154,524816,article,english american


In [307]:
sample_descriptions=sample_tags.groupby(["sample_id","category"])["tag_name"].agg(lambda d: ",".join(list(set(d)))).reset_index()
sample_descriptions.head()

,sample_id,category,tag_name
0,324,audio,"articulate,English US - General American,neutr..."
1,350,audio,"Educational,Calm,Approachable"
2,441,audio,"Announcer,English US - General American,neutra..."
3,499,audio,"English US - General American,authentic,neutra..."
4,964,audio,"Instructor,English US - General American,neutr..."


In [29]:
print(sample_attributes.shape)
print(sample_attributes.head())


(632084, 6)
   sample_id category pro_id  booking_score     attribute_name  \
0          7    audio     D6            0.0            purpose   
1          7    audio     D6            0.0        language_id   
2          7    audio     D6            0.0  gender_and_age_id   
3         21    audio   2362            0.0            purpose   
4         21    audio   2362            0.0        language_id   

    attribute_value  
0      phoneSystems  
1            eng-us  
2  youngAdultFemale  
3      phoneSystems  
4            eng-us  


In [ ]:
##join tags and samples and remove duplicated words 
##generate embeddings for descriptions and search terms

In [49]:
import spacy
from spacy.matcher import Matcher
nlp=spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": "hello"}, {"LOWER": "world"}]
matcher.add("pattern1", [pattern])
doc = nlp("uk voice over eng")
matches = matcher(doc)

    

In [82]:
def augment_language(sentence:str):
   finder=re.compile("(?i:eng|spa|ger|uk)")
   return re.findall(finder,sentence)

augment_language("uk voice over english")   

['uk', 'eng']

In [87]:
from sentence_transformers import SentenceTransformer
MODEL_NAME_HF="all-MiniLM-L12-v1"
model = SentenceTransformer(MODEL_NAME_HF)

Downloading: 100%|██████████| 737/737 [00:00<00:00, 736kB/s]
Downloading: 100%|██████████| 9.90k/9.90k [00:00<00:00, 6.60MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 116kB/s]
Downloading: 100%|██████████| 15.7k/15.7k [00:00<00:00, 15.7MB/s]
Downloading: 100%|██████████| 349/349 [00:00<00:00, 349kB/s]
Downloading: 100%|██████████| 134M/134M [00:04<00:00, 29.4MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 53.0kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 112kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 1.23MB/s]
Downloading: 100%|██████████| 352/352 [00:00<00:00, 352kB/s]
Downloading: 100%|██████████| 13.2k/13.2k [00:00<00:00, 13.2MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 700kB/s] 
Downloading: 100%|██████████| 190/190 [00:00<00:00, 190kB/s]


In [274]:
from sentence_transformers import util


def cosine_sim(vec1,vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

sentence1="british"
sentence2="united kingdom"
emb1=model.encode(sentence1)
emb2=model.encode(sentence2)

cosine_sim(emb1,emb2)

0.6134414

In [26]:
class SearchRecommender:
    
    def __init__(self,samples_filepath: str,pros_filepath: str,tags_filepath: str,separator=","):
        
        self.samples=pd.read_csv(samples_filepath,sep=separator)
        self.pros=pd.read_csv(pros_filepath,sep=separator)
        self.tags=pd.read_csv(tags_filepath,sep=separator)
        self.model=SentenceTransformer(MODEL_NAME_HF)


    @staticmethod
    def preprocess_data():
        pass
    
    def detect_intent(search: str):
        pass

    def suggest_samples(search: str,top_n: int=50):
        pass



    
        